In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.transform import rescale
from skimage import img_as_ubyte
import pickle
from sklearn.model_selection import train_test_split

In [0]:
#4/zQFDcqJxIv-ozKrojm6LD8PrmNVQi31nQcpfDqU51i3XWOlyL9wh1ZU
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/text-to-image")
os.listdir()

['model',
 'src-paola',
 'src-juan',
 'Flickr8k_Dataset',
 'data',
 'Flickr8k_text',
 'src-ely',
 'resultados_pasados',
 'pickles',
 'logs',
 'src-clean',
 'ToDo.gdoc',
 'my.png']

In [0]:
import pandas as pd
TOKEN_DIR = "./Flickr8k_text/"
# Load data
dir_token = TOKEN_DIR + "Flickr8k.lemma.token.txt"
df_lemma = pd.read_csv(dir_token, sep='\t', names=['ID', 'Text'], index_col=False)
print(df_lemma.shape)

# Remueve entries with nan values
df_lemma.dropna(inplace=True)
df_lemma.isnull().values.any()
df_lemma.columns = ['ID', 'lemma']
df_lemma

import pandas as pd
TOKEN_DIR = "./Flickr8k_text/"
# Load data
dir_token = TOKEN_DIR + "Flickr8k.token.txt"
df_token = pd.read_csv(dir_token, sep='\t', names=['ID', 'Text'], index_col=False)
print(df_token.shape)

# Remueve entries with nan values
df_token.dropna(inplace=True)
df_token.isnull().values.any()
df_token.head()

df_token.columns = ['ID', 'token']
df_token

df = df_token.merge(df_lemma, how='inner')
print(df.shape)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
#X_lemma_train, X_lemma_test, y_train, y_test = train_test_split(X_lemma, y, test_size=0.20, random_state=42)

(40460, 2)
(40460, 2)
(40460, 3)


In [0]:
mask_err = (~df['ID'].str.contains(".jpg.1#")).tolist()
df_corr = df[mask_err]

In [0]:
import pandas as pd

mask_err = (~df['ID'].str.contains(".jpg.1#")).tolist()
df_corr = df[mask_err]

mask_test = (df_corr['ID'].str.endswith('0')).tolist()
mask_train = (~df_corr['ID'].str.endswith('0')).tolist()
df_test_val = df_corr[mask_test]
df_train = df_corr[mask_train]

print(len(df_test_val))
print(len(df_train))

8091
32364


In [0]:
from math import ceil
test_size = len(df_test_val)
val_size = ceil(len(df_test_val)*0.10)

df_val = df_test_val[:val_size]
df_test = df_test_val[val_size:]

print(len(df_test))
print(len(df_val))

7281
810


In [0]:
df_val.ID
df_val.lemma
df_val.token

0       A child in a pink dress is climbing up a set o...
5              A black dog and a spotted dog are fighting
10      A little girl covered in paint sits in front o...
15      A man lays on a bench while his dog sits by him .
20         A man in an orange hat starring at something .
                              ...                        
4025                              A boy in a large pool .
4030    A woman in a purple shirt and a brown puppy si...
4035    A girl kisses her mother goodbye before she wa...
4040    A big brown dog floats in a canoe on the river...
4045       A brown and white animal surrounded by trees .
Name: token, Length: 810, dtype: object

In [0]:
from pickle import dump, load

with open("./pickles/PACO/val_id.pickle","wb") as f:
  pickle.dump(df_val.ID, f)

with open("./pickles/PACO/val_lemma.pickle","wb") as f:
  pickle.dump(df_val.lemma, f)

with open("./pickles/PACO/val_token.pickle","wb") as f:
  pickle.dump(df_val.token, f)


with open("./pickles/PACO/test_id.pickle","wb") as f:
  pickle.dump(df_test.ID, f)

with open("./pickles/PACO/test_lemma.pickle","wb") as f:
  pickle.dump(df_test.lemma, f)

with open("./pickles/PACO/test_token.pickle","wb") as f:
  pickle.dump(df_test.token, f)


with open("./pickles/PACO/train_id.pickle","wb") as f:
  pickle.dump(df_train.ID, f)

with open("./pickles/PACO/train_lemma.pickle","wb") as f:
  pickle.dump(df_train.lemma, f)

with open("./pickles/PACO/train_token.pickle","wb") as f:
  pickle.dump(df_train.token, f)

In [0]:
from pickle import dump

IMG_DIR = "./data/processed/normalized_64_complete/"
X_test = []

for image  in df_test.ID:
    try: 
        new_name = image[:-6] + '_' + image[-1] + '.jpg'
        I = imread(IMG_DIR + new_name)
        X_test.append(I)
    except Exception as error:
        print(error)
        print(image)
        pass

print(len(X_test))

from pickle import dump
dump(X_test, open("./pickles/PACO/X_test_64.pickle","wb"))

IMG_DIR = "./data/processed/normalized_64_complete/"
X_train = []

for image  in df_train.ID:
    try: 
        new_name = image[:-6] + '_' + image[-1] + '.jpg'
        I = imread(IMG_DIR + new_name)
        X_train.append(I)
    except Exception as error:
        print(error)
        print(image)
        pass

print(len(X_train))

from pickle import dump
dump(X_train, open("./pickles/PACO/X_train_64.pickle","wb"))

